# 01 - Basic Usage of Qurry

In Qurry, we use the instance of `Qurry` to store quantum circuits, experiments, build experiment, and excute them on local simulator or pending to the real quantum machine on `IBM`. In this chapter (or we say this jupyter file), we will introduce how to use Qurry to build a simple quantum circuit and measure their Renyi Entropy on local simulator.


If you are using Colab, you can run the following command in terminal to install Qurry.

```bash
pip install qurrium
```


In [1]:
from IPython.display import display
from qurry import __version__

print(f"| Current Version: {__version__}")

| Current Version: 0.9.2.dev1


## 1.1 Setup Environment and Creat experiment executor


In [2]:
from qurry import EntropyMeasure
from qiskit import QuantumCircuit

- Import simulator


In [3]:
from qiskit.providers.basic_provider import BasicProvider

basic_provider = BasicProvider()
backend_sim = basic_provider.get_backend("basic_simulator")

- Next, we will initialize our `EntropyMeasure` instance. There are two methods implemented based on two theories. The defaul method is `randomized` based on Randomized Measure. The other method is `hadamard` based on Hadamard Test. 

- In the following tutorial, we will use `randomized` as our measurement method.


In [4]:
experiment_hadamard = EntropyMeasure(method="hadamard")
print('| The first method is which is `Hadamard Test`".')
experiment_randomized = EntropyMeasure()
print("| The default method is which is `Randomized Measure` we will use.")

| The first method is which is `Hadamard Test`".
| The default method is which is `Randomized Measure` we will use.


---

## 1.2 Load quantum circuits

- In `Qurry`, there are built-in quantum circuits that construct certain wave functions, or equivalently, quantum states. They can be imported from `qurry.recipe`.
- The user can define customized circuits following Qiskit workflow.



In [5]:
from qurry.recipe import TrivialParamagnet, GHZ

In [6]:
sample01 = TrivialParamagnet(8)
print("| trivial paramagnet in 8 qubits:")
print(sample01)

| trivial paramagnet in 8 qubits:
     ┌───┐
q_0: ┤ H ├
     ├───┤
q_1: ┤ H ├
     ├───┤
q_2: ┤ H ├
     ├───┤
q_3: ┤ H ├
     ├───┤
q_4: ┤ H ├
     ├───┤
q_5: ┤ H ├
     ├───┤
q_6: ┤ H ├
     ├───┤
q_7: ┤ H ├
     └───┘


In [7]:
sample02 = GHZ(8)
print("| GHZ in 8 qubits:")
print(sample02)

| GHZ in 8 qubits:
     ┌───┐                                   
q_0: ┤ H ├──■────────────────────────────────
     └───┘┌─┴─┐                              
q_1: ─────┤ X ├──■───────────────────────────
          └───┘┌─┴─┐                         
q_2: ──────────┤ X ├──■──────────────────────
               └───┘┌─┴─┐                    
q_3: ───────────────┤ X ├──■─────────────────
                    └───┘┌─┴─┐               
q_4: ────────────────────┤ X ├──■────────────
                         └───┘┌─┴─┐          
q_5: ─────────────────────────┤ X ├──■───────
                              └───┘┌─┴─┐     
q_6: ──────────────────────────────┤ X ├──■──
                                   └───┘┌─┴─┐
q_7: ───────────────────────────────────┤ X ├
                                        └───┘


In [8]:
sample03 = QuantumCircuit(8)
sample03.x(range(0, 8, 2))
print("| Custom circuit:")
print(sample03)

| Custom circuit:
     ┌───┐
q_0: ┤ X ├
     └───┘
q_1: ─────
     ┌───┐
q_2: ┤ X ├
     └───┘
q_3: ─────
     ┌───┐
q_4: ┤ X ├
     └───┘
q_5: ─────
     ┌───┐
q_6: ┤ X ├
     └───┘
q_7: ─────
          


- After we prepare three quantum circuits, we append them to the `EntropyMeasure` object by the `.add()` method.
- The attribute `.waves` of `Qurry` object is a customized `dict`. we can check the stored circuits by printing the dictionary.

In [9]:
experiment_randomized.add(
    wave=sample01,  # The circuit object
    key="TrivialParamagnet",  # The name of the circuit
    replace=False,  # True for if you want to replace the circuit with the same name.
)  # Default is False, so if you add the same name when False,
# it will be added with serial number.
experiment_randomized.add(sample02, "GHZ")
experiment_randomized.add(sample03)
print(experiment_randomized.waves)

{'TrivialParamagnet': <qurry.recipe.simple.paramagnet.TrivialParamagnet object at 0x78d22b5c1880>, 'GHZ': <qurry.recipe.simple.cat.GHZ object at 0x78d228b36060>, 2: <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x78d231fda630>}


If you do not give the name of circuit, it will be named by series number.

Now, waves are loading, time to excute.


## 1.3 Excute Circuits


- We can use `.measure()` method to excute the specific circuit in the `EntropyMeasure` object. The `.measure()` method  returns a hash id of this experiment and save a `Experiment` object in `EntropyMeasure`.
- `EntropyMeasure.measure()` requires the following input parameters:
  - wave: the name of circuit in `Qurry.waves` object
  - times: the number of random measurements
  - shots: the number of shots for each measurement. The default is 1024.


In [10]:
exp1 = experiment_randomized.measure(
    wave="TrivialParamagnet", times=100, shots=1024, backend=backend_sim
)
exp1

'617d627d-9cb5-473f-9198-3b3b61cf1fa7'

- You can specify the backend by add a parameter `backend` in `measure` method, if you do not specify, it will use simulator as the default backend. Moreover, qiskit has multiple simulators, they are `QasmSimulatorPy` from `qiskit`, `AerSimulator` from `qiskit-aer`, and `AerSimulator` from `qiskit-aer-gpu`. Qurry will check the availability of these simulators and use the first available one as the default backend in following order:
  1. `AerSimulator` from `qiskit-aer-gpu`
  2. `AerSimulator` from `qiskit-aer`
  3. `QasmSimulatorPy` from `qiskit`


In [11]:
exp2 = experiment_randomized.measure(
    wave="TrivialParamagnet",
    times=100,
    shots=1024,
)
exp2

'ea7980d4-7cf5-4250-93c1-34ec870cf26f'

- And we can check the attribute `.exps` of `EntropyMeasure` object, it will return the hash id of all experiments.


In [12]:
print(experiment_randomized.exps)

{'617d627d-9cb5-473f-9198-3b3b61cf1fa7': <EntropyMeasureRandomizedExperiment(exp_id=617d627d-9cb5-473f-9198-3b3b61cf1fa7, EntropyMeasureRandomizedArguments(exp_name='experiment.N_U_100.qurrent_randomized', times=100, qubits_measured=[0, 1, 2, 3, 4, 5, 6, 7], registers_mapping={0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7}, actual_num_qubits=8, unitary_located=[0, 1, 2, 3, 4, 5, 6, 7], random_unitary_seeds=None, workers_num=16), Commonparams(exp_id='617d627d-9cb5-473f-9198-3b3b61cf1fa7', target_keys=['TrivialParamagnet'], shots=1024, backend=<qiskit.providers.basic_provider.basic_simulator.BasicSimulator object at 0x78d23c1fe060>, run_args={}, transpile_args={}, tags=(), default_analysis=[], save_location=PosixPath('.'), filename='', files={}, serial=None, summoner_id=None, summoner_name=None, datetimes=DatetimeDict({'build': '2024-11-04 06:53:10', 'run.001': '2024-11-04 06:53:10'})), unused_args_num=0, analysis_num=0)>, 'ea7980d4-7cf5-4250-93c1-34ec870cf26f': <EntropyMeasureRandomize

- The following is the other informatiom you can access.


In [13]:
experiment_randomized.waves

WaveContainer({
  'TrivialParamagnet': <qurry.recipe.simple.paramagnet.TrivialParamagnet object at 0x78d22b5c1880>,
  'GHZ': <qurry.recipe.simple.cat.GHZ object at 0x78d228b36060>,
  2: <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x78d231fda630>})

In [14]:
experiment_randomized.exps[exp1]

<EntropyMeasureRandomizedExperiment(exp_id=617d627d-9cb5-473f-9198-3b3b61cf1fa7, 
  EntropyMeasureRandomizedArguments(exp_name='experiment.N_U_100.qurrent_randomized', times=100, qubits_measured=[0, 1, 2, 3, 4, 5, 6, 7], registers_mapping={0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7}, actual_num_qubits=8, unitary_located=[0, 1, 2, 3, 4, 5, 6, 7], random_unitary_seeds=None, workers_num=16),
  Commonparams(exp_id='617d627d-9cb5-473f-9198-3b3b61cf1fa7', target_keys=['TrivialParamagnet'], shots=1024, backend=<qiskit.providers.basic_provider.basic_simulator.BasicSimulator object at 0x78d23c1fe060>, run_args={}, transpile_args={}, tags=(), default_analysis=[], save_location=PosixPath('.'), filename='', files={}, serial=None, summoner_id=None, summoner_name=None, datetimes=DatetimeDict({'build': '2024-11-04 06:53:10', 'run.001': '2024-11-04 06:53:10'})),
  unused_args_num=0,
  analysis_num=0))>

In [15]:
print(experiment_randomized.exps[exp1].statesheet())

 # EntropyMeasureRandomizedExperiment with exp_id=617d627d-9cb5-473f-9198-3b3b61cf1fa7
 - arguments
   - exp_name --------------------- experiment.N_U_100.qurrent_randomized
   - times ------------------------ 100
   - qubits_measured -------------- [0, 1, 2, 3, 4, 5, 6, 7]
   - registers_mapping ------------ {0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7}
   - actual_num_qubits ------------ 8
   - unitary_located -------------- [0, 1, 2, 3, 4, 5, 6, 7]
   - random_unitary_seeds --------- None
   - workers_num ------------------ 16
 - commonparams
   - exp_id ----------------------- 617d627d-9cb5-473f-9198-3b3b61cf1fa7   # This is ID is generated by Qurry which is different from 'job_id' for pending.
   - target_keys ------------------ ['TrivialParamagnet']
   - shots ------------------------ 1024
   - backend ---------------------- <qiskit.providers.basic_provider.basic_simulator.BasicSimulator object at 0x78d23c1fe060>
   - run_args --------------------- {}
   - transpile_args -----

- Then, there is a experiment completed, and we can calculate the Renyi Entropy by `analyze`.


---

## 1.4 Post-Processing

- To obtain the Renyi Entropy of the whole system and the subsystems. We use `exps[].analyze()` method to calculate and store a `Analysis` object in `Experiment`.
- The `analyze` method requires the following input parameters:
  - degree: The size or selected qubits of the subsystem. Please refer to the following chart for the selection.


### Qubit Selction Hint

For example, consider a system with 12 qubits indexed as `[0, 1, ... 11]`.

Entering an index and running `.analyze` will select the bitstring corresponding to the chosen qubit.

In [16]:
exp1_report1 = experiment_randomized.exps[exp1].analyze(selected_qubits=[3, 4])
# The subsystem of qubits 3 and 4 is selected for the analysis.
exp1_report2 = experiment_randomized.exps[exp1].analyze(selected_qubits=[0])
# The subsystem of qubit 0 is selected for the analysis.

|  - 00:00 < ?

|  - 00:00 < ?

- The `analyze` will return the result of this analysis.


In [17]:
exp1_report2.export()

({'purity': 1.0357794189453124,
  'entropy': -0.050716797351889734,
  'puritySD': 0.4280723538099501,
  'entropySD': 0.5962445774527289,
  'num_classical_registers': 8,
  'classical_registers': [0],
  'classical_registers_actually': [0],
  'all_system_source': "AnalysisHeader(serial=0, datetime='2024-11-04 06:53:12', summoner=None, log={})",
  'purityAllSys': 1.0216295623779297,
  'entropyAllSys': -0.030872177294001345,
  'puritySDAllSys': 0.8338110453178725,
  'entropySDAllSys': 1.1774669649521534,
  'num_classical_registers_all_sys': 8,
  'classical_registers_all_sys': None,
  'classical_registers_actually_all_sys': [0, 1, 2, 3, 4, 5, 6, 7],
  'errorRate': -0.010798884146186752,
  'mitigatedPurity': 1.0243925574850874,
  'mitigatedEntropy': -0.03476867647991984,
  'counts_num': 100,
  'taking_time': 0.000274821,
  'taking_time_all_sys': 0.004912981,
  'counts_used': None,
  'input': {'num_qubits': 8,
   'selected_qubits': [0],
   'registers_mapping': {0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5:

In [18]:
print(exp1_report1)
print(exp1_report1.header)

qurrentRandomized.Analysis with serial=0, AnalysisInput(num_qubits=8, selected_qubits=[3, 4], registers_mapping={0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7}, shots=1024, unitary_located=[0, 1, 2, 3, 4, 5, 6, 7]), AnalysisContent(purity=0.8980053901672364, entropy=0.1552039902948788, and others), 0 unused arguments
AnalysisHeader(serial=0, datetime='2024-11-04 06:53:12', summoner=None, log={})


- It can also be found in the attribute `.reports` of `Experiment` object.


In [19]:
print(experiment_randomized.exps[exp1].reports)
print(experiment_randomized.exps[exp1].reports[0])
print(experiment_randomized.exps[exp1].reports[0].header)

AnalysisContainer(length=2, {0{...}, 1{...}})
qurrentRandomized.Analysis with serial=0, AnalysisInput(num_qubits=8, selected_qubits=[3, 4], registers_mapping={0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7}, shots=1024, unitary_located=[0, 1, 2, 3, 4, 5, 6, 7]), AnalysisContent(purity=0.8980053901672364, entropy=0.1552039902948788, and others), 0 unused arguments
AnalysisHeader(serial=0, datetime='2024-11-04 06:53:12', summoner=None, log={})


- We can use `.export()` to get the result of this analysis.


In [20]:
main, side_prodct = experiment_randomized.exps[exp1].reports[0].export()

- And there is it


In [21]:
main

{'purity': 0.8980053901672364,
 'entropy': 0.1552039902948788,
 'puritySD': 0.527134871683887,
 'entropySD': 0.8468711586646002,
 'num_classical_registers': 8,
 'classical_registers': [3, 4],
 'classical_registers_actually': [3, 4],
 'all_system_source': 'independent',
 'purityAllSys': 1.0216295623779297,
 'entropyAllSys': -0.030872177294001345,
 'puritySDAllSys': 0.8338110453178725,
 'entropySDAllSys': 1.1774669649521534,
 'num_classical_registers_all_sys': 8,
 'classical_registers_all_sys': None,
 'classical_registers_actually_all_sys': [0, 1, 2, 3, 4, 5, 6, 7],
 'errorRate': -0.010798884146186752,
 'mitigatedPurity': 0.8842334024006319,
 'mitigatedEntropy': 0.17750086097138182,
 'counts_num': 100,
 'taking_time': 0.000601532,
 'taking_time_all_sys': 0.004912981,
 'counts_used': None,
 'input': {'num_qubits': 8,
  'selected_qubits': [3, 4],
  'registers_mapping': {0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7},
  'shots': 1024,
  'unitary_located': [0, 1, 2, 3, 4, 5, 6, 7]},
 'heade

- Also, `sideProdct` will record the data of each circuit, and you can use it to do more analysis.


In [22]:
side_prodct

{'purityCells': {88: 0.7537631988525391,
  23: 0.8546485900878906,
  61: 0.5635795593261719,
  45: 0.5866680145263672,
  78: 0.9752464294433594,
  37: 0.4576873779296875,
  80: 1.5215873718261719,
  86: 0.42473411560058594,
  48: 0.8972320556640625,
  90: 1.5043869018554688,
  26: 0.7555198669433594,
  49: 0.7371978759765625,
  9: 1.925821304321289,
  11: 0.3872661590576172,
  54: 0.30536651611328125,
  81: 1.4056129455566406,
  77: 0.3289756774902344,
  40: 0.2655467987060547,
  84: 0.292938232421875,
  29: 0.8840713500976562,
  55: 1.6578865051269531,
  50: 1.4195060729980469,
  73: 1.5754203796386719,
  16: 0.4380607604980469,
  10: 2.2353553771972656,
  30: 0.26279449462890625,
  7: 1.7535362243652344,
  47: 0.8102226257324219,
  53: 0.8633518218994141,
  70: 1.4246673583984375,
  63: 1.0527515411376953,
  74: 0.8632659912109375,
  20: 1.8426628112792969,
  43: 0.5590476989746094,
  46: 1.595367431640625,
  56: 2.3237838745117188,
  42: 0.9213962554931641,
  19: 1.7795829772949219,

In [23]:
experiment_randomized.exps[exp1].reports[0].statesheet()

 # qurrentRandomized.Analysis with serial=0
 - header
   - serial -------------------------------------- 0
   - datetime ------------------------------------ 2024-11-04 06:53:12
   - summoner ------------------------------------ None
   - log ----------------------------------------- {}
 - input
   - num_qubits ---------------------------------- 8
   - selected_qubits ----------------------------- [3, 4]
   - registers_mapping --------------------------- {0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7}
   - shots --------------------------------------- 1024
   - unitary_located ----------------------------- [0, 1, 2, 3, 4, 5, 6, 7]
 - outfields ----------------------------------- 0 ........  # Number of unused arguments.
 - content
   - purity -------------------------------------- 0.8980053901672364
   - entropy ------------------------------------- 0.1552039902948788
   - puritySD ------------------------------------ 0.527134871683887
   - entropySD ----------------------------------

---

## Post-Process Availablities and Version Info

We currently do not support Qsikit 1.0 for we are working on it, and we will support it in the future.


In [24]:
from qurry.process import AVAIBILITY_STATESHEET

print(AVAIBILITY_STATESHEET)

 | Qurry version: 0.9.2.dev1
--------------------------------------------------------
 ### Qurry Post-Processing
   - Backend Availability ................... Python Cython Rust  
 - randomized_measure
   - entangled_entropy.entropy_core_2 ....... Yes    Depr.  Yes   
   - entangle_entropy.purity_cell_2 ......... Yes    Depr.  Yes   
   - wavefunction_overlap ................... Yes    Depr.  Yes   
   - echo_cell .............................. Yes    Depr.  Error 
 - utils
   - randomized ............................. Yes    Depr.  Yes   
   - construct .............................. Yes    No     Yes   
   - dummy .................................. Yes    No     Yes   
 - hadamard_test
   - purity_echo_core ....................... Yes    No     Yes   
 - magnet_square
   - magnsq_core ............................ Yes    No     No    
--------------------------------------------------------
   + Yes ...... Working normally.
   + Error .... Exception occurred.
   + No ....... Not suppo

In [25]:
from qurry.tools.qiskit_version import QISKIT_VERSION_STATESHEET

print(QISKIT_VERSION_STATESHEET)

 | Qurry version: 0.9.2.dev1
--------------------------------------------
 ### Qiskit version
 - main
   - qiskit-aer .............. 0.15.0
   - qiskit-aer-gpu .......... 0.14.2
   - qiskit-ibm-runtime ...... 0.30.0
 - deprecated
 - into-community
--------------------------------------------
 + qiskit-aer-gpu should have the same version as qiskit-aer.
--------------------------------------------

